# Optimization methods
Optimization methods attempt to find the minimum or maximum of functions. Here the steepest descent, conjugate gradient, Newton-Raphson, and Broyden-Fletcher-Goldfarb-Shanno methods are presented.

For all methods we will follow generally the same algorithm:
```
define and initial starting point:
while converged = False & iteration < ITMAX
    define the step direction
    take a step in that direction
    if convergence criteria met:
        exit loop
```
Between all of the different methods presented in this notebook, they only change in defining the step direction. Alternatively, one thing that will stay consistent between all methods is the convergence criteria. An optimization algorithm should always test three convergence criteria
1. update: to ensure we're not taking too big of a step.
2. residual: to see if solution estimates are within errors of eachother
3. iteration: set an iteration maximum so you do not get caught in an infinite loop if function is ilbehaved. Additionally this save you from crashing your computer if you're writing code yourself!

# 4.1 Steepest Descent #

The steepest descent method is the simplest iterative method of optimization. Steepest descent makes iterative steps in the direction of greatest decrease.

First, an initial guess,$x_0$, is taken. The starting search direction, $sd_0$, is taken as the direction of steepest descent.

 $$ sd_0 = -\nabla f(x_0)$$

A step is taken by using a line search minimization along this search direction, 

$$ x_i = x_0 + \lambda sd_0 $$

where $\lambda$ minimizes $f(x)$ in the search direction.

A new search direction is constructed as the direction of steepest descent at the new point.

 $$ sd_i = -\nabla f(x_i)$$
 
 
 The cell directly below implements steepest descent on the function,
 $$ f(x,y) = sin(0.5x^2-0.25y^2+3)*cos(2x+1-e^y)$$
Following the code is an image depicting steepest descent implemented on this function.


In [ ]:
###########
# IMPORTS #
###########
import numpy as np
from scipy.optimize import line_search as linesearch
import scipy.linalg as spla
import scipy as sp
import plotting_functions as pf
import matplotlib.pyplot as plt
%matplotlib inline
#############
# FUNCTIONS #
#############


def f(point):
    x = point[0]
    y = point[1]
    return np.sin(0.5 * x**2 - 0.25 * y**2 + 3) * np.cos(2 * x + 1 - np.exp(y))


def partial_f_partial_x(x, y):
    part1 = x * np.cos(0.5 * x**2 - 0.25 * y**2 + 3) * \
        np.cos(2 * x + 1 - np.exp(y))
    part2 = -2 * np.sin(0.5 * x**2 - 0.25 * y**2 + 3) * \
        np.sin(2 * x + 1 - np.exp(y))
    return part1 + part2


def partial_f_partial_y(x, y):
    part1 = np.exp(y) * np.sin(0.5 * x**2 - 0.25 * y**2 + 3) * \
        np.sin(2 * x + 1 - np.exp(y))
    part2 = -0.5 * y * \
        np.cos(0.5 * x**2 - 0.25 * y**2 + 3) * np.cos(2 * x + 1 - np.exp(y))
    return part1 + part2


def Gradient(point):
    x = point[0]
    y = point[1]
    return np.array([partial_f_partial_x(x, y), partial_f_partial_y(x, y)])


def partial_sq_f_partial_x_sq(x, y):
    part1 = -4 * x * \
        np.cos(0.5 * x**2 - 0.25 * y**2 + 3) * np.sin(2 * x + 1 - np.exp(y))
    part2 = -4 * np.sin(0.5 * x**2 - 0.25 * y**2 + 3) * \
        np.cos(2 * x + 1 - np.exp(y))
    part3 = np.cos(2 * x + 1 - np.exp(y)) * (np.cos(0.5 * x**2 -
                                                    0.25 * y**2 + 3) - (x**2) * np.sin(0.5 * x**2 - 0.25 * y**2 + 3))
    return part1 + part2 + part3


def partial_sq_f_partial_y_sq(x, y):
    part1 = -np.exp(y) * y * np.cos(0.5 * x**2 - 0.25 * y **
                                    2 + 3) * np.sin(2 * x + 1 - np.exp(y))
    part2 = np.sin(0.5 * x**2 - 0.25 * y**2 + 3) * ((np.exp(y) * np.sin(2 *
                                                                        x + 1 - np.exp(y))) - (np.exp(2 * y) * np.cos(2 * x + 1 - np.exp(y))))
    part3 = np.cos(2 * x + 1 - np.exp(y)) * ((-0.25 * y**2 * np.sin(0.5 *
                                                                    x**2 - 0.25 * y**2 + 3)) - (0.5 * np.cos(0.5 * x**2 - 0.25 * y**2 + 3)))
    return part1 + part2 + part3


def partial_sq_f_partial_x_y(x, y):
    part1 = (x * np.exp(y)) * np.cos(0.5 * x**2 - 0.25 * y**2 + 3) * \
        np.sin(2 * x + 1 - np.exp(y))
    part2 = y * np.cos(0.5 * x**2 - 0.25 * y**2 + 3) * \
        np.sin(2 * x + 1 - np.exp(y))
    part3 = (2 * np.exp(y)) * np.sin(0.5 * x**2 - 0.25 * y**2 + 3) * \
        np.cos(2 * x + 1 - np.exp(y))
    part4 = (0.5 * x * y) * np.sin(0.5 * x**2 - 0.25 * y**2 + 3) * \
        np.cos(2 * x + 1 - np.exp(y))
    return part1 + part2 + part3 + part4


def Hessian(point):
    x = point[0]
    y = point[1]
    part1 = partial_sq_f_partial_x_sq(x, y)
    part2 = partial_sq_f_partial_x_y(x, y)
    part3 = partial_sq_f_partial_y_sq(x, y)
    return np.array([[part1, part2], [part2, part3]])
# If you don't have analytical gradients... Use numerical gradients! 
# def Central_Diff_X(a,y,h):
#     return (f((a+h,y)) - f((a-h,y))) / (2*h)
# def Central_Diff_Y(x,b,h):
#     return (f((x,b+h)) - f((x,b-h))) / (2*h)
# def Gradient(point):
#     x= point[0]
#     y = point[1]
#     step = 1e-6
#     return np.array([Central_Diff_X(x,y,step),Central_Diff_Y(x,y,step)])
# def Central_Diff_2nd_X(a,y,h):
# Approximation by Central Difference Method
#         return (f((a+h,y)) - 2*f((a,y)) + f((a-h,y))) / (h**2)
# def Central_Diff_2nd_Y(x,b,h):
# Approximation by Central Difference Method
#         return (f((x,b+h)) - 2*f((x,b)) + f((x,b-h))) / (h**2)
# def Central_Diff_2nd_XY(a,b,h):
# Approximation by Central Difference Method
#         return (-1.0/(2*h**2))*(f((a+h,b))+f((a-h,b))+f((a,b+h))+f((a,b-h))-(2*f((a,b)))-f((a+h,b+h))-f((a-h,b-h)))
# def Hessian(a,b):
#     step = 1e-6
#     return np.array([[Central_Diff_2nd_X(a,b,step),Central_Diff_2nd_XY(a,b,step)],[Central_Diff_2nd_XY(a,b,step),Central_Diff_2nd_Y(a,b,step)]])
##########
# GLOBAL VARS
##########
initial_guess = np.array((1.6, -1.5))  # initial guess
#convergence criteria
residual_tolerance = 1e-6  
update_tolerance = 1e-5
IT_MAX = 1000
# domain for plotting
xminimum = -1
xmaximum = 2
yminimum = -4
ymaximum = 1
steps_per_method = dict()
difference_per_method = dict()


In [ ]:
##########
#MAIN CODE
##########
old_guess = initial_guess
search_direction = -Gradient(old_guess) # set the search direction to the negative gradient at the initial guess
converged = False 
num_steps = 0
steps_x =[]
steps_y =[]
steps_x.append(old_guess[0])
steps_y.append(old_guess[1])
#repeat until convergenced or exceed IT_MAX
while not converged and num_steps <= IT_MAX: 
    print("STEP {:d}".format(num_steps))
    num_steps+=1
    gamma = linesearch(f,Gradient,old_guess,search_direction)[0] # find the step size that minimizes f along the search direction
    if gamma == None: 
        print("ERROR: No appropriate step size found. Exiting minimization")
        exit()
    else:
        new_guess = old_guess + gamma*search_direction # update the current guess
        search_direction = -Gradient(new_guess) # set the new search direction to the negative gradient at the new guess
        diff = old_guess-new_guess # calculate the vector difference of the old guess and new guess
        diff = np.sqrt(np.dot(diff,diff)) # calculate the magnitude of the separation between the old guess and the new guess
        print("OLD GUESS: {}".format(old_guess))
        print("NEW GUESS: {}".format(new_guess))
        print("DIFFERENCE: {}".format(diff))
        old_guess = new_guess # update the old guess so we can take another step
        steps_x.append(old_guess[0])
        steps_y.append(old_guess[1])
        if(diff<residual_tolerance): # if the step was small enough consider this the minimum
            converged=True
steps_x = np.array(steps_x)
steps_y = np.array(steps_y)
steps_per_method["Steepest Descent"] = num_steps
difference_per_method["Steepest Descent"] = diff


In [ ]:
pf.plot_func(xminimum, xmaximum, yminimum, ymaximum, steps_x, steps_y, f)

# 4.2 Conjugate Gradient #

Many similarities can be drawn with the steepest descent method, however the conjugate gradient attempts to ensure previous progress is not undone by the next steps. This restriction is called *conjugacy*. 


First an initial guess, $x_0$, is taken. The starting search direction, $sd_0$ is taken as the direction of steepest descent.

 $$ sd_0 = -\nabla f(x_0)$$

A step is taken by using a line search minimization along this search direction.

A new search direction is constructed such that successive steps don't undo the progress of prior steps.

This is done through a process similar to orthogonalization. 

$$ sd_{i+1} = -\nabla f(x_{i+1}) + \frac{(-\nabla f(x_{i+1})) \bullet (-\nabla f(x_{i+1})) }{(-\nabla f(x_{i})) \bullet (-\nabla f(x_{i}))} sd_{i} $$

In practice, the line search minimization may fail because the search direction may point uphill.

This can be resolved by reseting the search direction to point in the direction of steepest descent.

The code below applies the conjugate gradient method to the example function and plots the optimization path.


In [ ]:
#############
# MAIN CODE #
#############
converged = False
num_steps = 0
old_guess = initial_guess
old_grad = -Gradient(old_guess)
# set first search direction in the direction of steepest descent
old_search_direction = old_grad
steps_x = []
steps_y = []
steps_x.append(old_guess[0])
steps_y.append(old_guess[1])
while not converged and num_steps <= IT_MAX:
    print("STEP {:d}".format(num_steps))
    alpha = linesearch(f, Gradient, old_guess, old_search_direction)[0]
    if alpha == None:
        print("WARNING: LINE SEARCH DID NOT CONVERGE... RESETTING SEARCH DIRECTION TO STEEPEST DESCENT!")
        old_search_direction = -Gradient(old_guess)
        alpha = linesearch(f, Gradient, old_guess, old_search_direction)[0]  # step size
    new_guess = old_guess + alpha * old_search_direction
    new_grad = -Gradient(new_guess)
    # gamma encompasses the fraction in the above equation
    gamma = np.dot(new_grad - old_grad, new_grad) / np.dot(old_grad, old_grad)
    # update the search direction
    new_search_direction = new_grad + np.dot(gamma, old_search_direction)
    diff = old_guess - new_guess
    diff = np.sqrt(np.dot(diff, diff))
    print("OLD GUESS: {}".format(old_guess))
    print("NEW GUESS: {}".format(new_guess))
    print("DIFFERENCE: {}".format(diff))
    num_steps += 1
    old_guess = new_guess
    old_grad = new_grad
    old_search_direction = new_search_direction
    steps_x.append(old_guess[0])
    steps_y.append(old_guess[1])
    if(diff < residual_tolerance):
        converged = True
steps_x = np.array(steps_x)
steps_y = np.array(steps_y)
steps_per_method["Conjugate Gradient"] = num_steps
difference_per_method["Conjugate Gradient"] = diff


In [ ]:
pf.plot_func(xminimum, xmaximum, yminimum, ymaximum, steps_x, steps_y, f)

# 4.3 Newton-Raphson

In the root-finding section of these lessons, the Newton-Raphson root finding method was described for a one-dimensional system. This method can be applied to higher dimensions by simple modifications.
As a reminder, the Newton-Raphson method takes iterative steps:
$$x_{i+1}\ =\ x_i - \frac{f(x_i)}{f'(x_i)}$$

To minimize the first derivative to find an extremum, the following iterative equation could be used:
$$x_{i+1}\ =\ x_i - \frac{f'(x_i)}{f''(x_i)}$$

To optimize a higher order function, the Hessian matrix is introduced.
The Hessian is a square matrix composed of second derivatives that describe the local curvature of a function. The iterative steps are given by:
$$ p_i = (x_i,y_i) $$
and
$$p_{i+1} = p_i - \gamma[H(p_i)]^{-1}]\nabla f(p_i)$$

where the step size, $\gamma$, is found using a line search and $H(p_i)$ is the Hessian:
$$H(p_i)= \ \begin{bmatrix}
{\frac{\partial^2 f(x_i,y_i)}{\partial x^2}} 
& {\frac{\partial^2 f(x_i,y_i)}{\partial x \partial y}} \\
{\frac{\partial^2 f(x_i,y_i)}{\partial y \partial x}} 
& {\frac{\partial^2 f(x_i,y_i)}{\partial y^2}} 
\end{bmatrix} $$


It is important to note that the Newton-Raphson method requires the Hessian to be positive definite. A positive definite Hessian ensures a local minimum near $p_i$ is isolated and not a saddle point or local maximum. We will see below that the Newton-Raphson method finds the minimum of a well-behaved, convex function, such as $x^2\ +\ y^2$. If we solve for the Hessian of this function:
$$Hf(x,y)= \ \begin{bmatrix}
{\frac{\partial^2 f(x,y)}{\partial x^2}} 
& {\frac{\partial^2 f(x,y)}{\partial x \partial y}} \\
{\frac{\partial^2 f(x,y)}{\partial y \partial x}} 
& {\frac{\partial^2 f(x,y)}{\partial y^2}} 
\end{bmatrix} $$
$$Hf(x,y)= \ \begin{bmatrix}
{\frac{\partial^2 (x^2+y^2)}{\partial x^2}} 
& {\frac{\partial^2 (x^2+y^2)}{\partial x \partial y}} \\
{\frac{\partial^2 (x^2+y^2)}{\partial y \partial x}} 
& {\frac{\partial^2 (x^2+y^2)}{\partial y^2}} 
\end{bmatrix} $$

$$Hf(x,y)= \ \begin{bmatrix}
2
& 0 \\
0 
& 2 
\end{bmatrix} $$
These postive values of the Hessian indicate that our function has a local minimum, thus Newton's method of optimization will perform well.
Despite the requirement of a positive definite Hessian, the Newton-Raphson's method has excellent convergence since most functions near a minimum resemble $x^2\ +\ y^2$.

The code below applies the Newton-Raphson optimization method to the example function and plots the optimization path.

In [ ]:
##########
# MAIN CODE
##########
old_guess = initial_guess
old_search_direction = -Gradient(old_guess)
num_steps = 0
converged = False
steps_x = []
steps_y = []
steps_x.append(old_guess[0])
steps_y.append(old_guess[1])
while not converged and num_steps <= IT_MAX:
    print("STEP {:d}".format(num_steps))
    num_steps += 1
    A = Hessian(old_guess)
    try:
        if num_steps == 1:
            new_search_direction = -Gradient(old_guess)
            gamma = linesearch(
                f, Gradient, old_guess, new_search_direction)[0]  # step size
            new_guess = old_guess + gamma * new_search_direction
        else:
            new_search_direction = spla.solve(A, -Gradient(old_guess))
            new_guess = old_guess + new_search_direction
    except:
        print("WARNING: HESSIAN IS NOT POSITIVE DEFINITE... RESETTING SEARCH DIRECTION TO STEEPEST DESCENT!")
        new_search_direction = -Gradient(old_guess)
        gamma = linesearch(f, Gradient, old_guess, new_search_direction)[
            0]  # step size
        new_guess = old_guess + gamma * new_search_direction
    diff = old_guess - new_guess
    diff = np.sqrt(np.dot(diff, diff))
    print("OLD GUESS: {}".format(old_guess))
    print("NEW GUESS: {}".format(new_guess))
    print("DIFFERENCE: {}".format(diff))
    old_guess = new_guess
    steps_x.append(old_guess[0])
    steps_y.append(old_guess[1])
    old_search_direction = new_search_direction
    if(diff < residual_tolerance):
        converged = True
steps_x = np.array(steps_x)
steps_y = np.array(steps_y)
steps_per_method["Newton-Raphson"] = num_steps
difference_per_method["Newton-Raphson"] = diff


In [ ]:
pf.plot_func(xminimum, xmaximum, yminimum, ymaximum, steps_x, steps_y, f)

# 4.4 Broyden-Fletcher-Goldfarb-Shanno (BFGS)

   The BFGS method is one of the quasi-Newton methods and, as the name suggests, is an approximation of the Newton optimization method discussed above. The major difference of these methods compared to the Newton method is the Hessian is not calculated directly, but instead approximated. This approximation results in the BFGS method being more robust and often times converging more quickly than the Newton method. 
   
 The only difference to the Newton method is the update of the approximate Hessian, $H_{approx}$ and the update of the search direction. The mathematics of these updates are out of the scope of these notebooks; however, the interested reader is directed towards *Polak, E. Computational Methods in Optimization; a Unified Approach. New York: Academic, 1971. Print.* for a more detailed explanation.
 
 The code below applies the BFGS method to the example function and plots the optimization path.

In [ ]:
##########
# MAIN CODE
##########
old_guess = initial_guess

approx_hessian = np.identity(2)
inverse_approx_hessian = np.identity(2)

num_steps = 0

s_i = np.array([1.0, 0.0])
y_i = np.array([0.0, 1.0])

converged = False
steps_x = []
steps_y = []
steps_x.append(old_guess[0])
steps_y.append(old_guess[1])
while not converged and num_steps <= IT_MAX:
    print("STEP {:d}".format(num_steps))
    num_steps += 1
    new_search_direction = np.dot(inverse_approx_hessian, -Gradient(old_guess))
    try:
        s_i = linesearch(f, Gradient, old_guess, new_search_direction)[
            0] * new_search_direction
    except:
        print("WARNING: HESSIAN IS NOT POSITIVE DEFINITE... RESETTING SEARCH DIRECTION TO STEEPEST DESCENT!")
        new_search_direction = -Gradient(old_guess)
        s_i = linesearch(f, Gradient, old_guess, new_search_direction)[
            0] * new_search_direction

    new_guess = old_guess + s_i
    y_i = Gradient(new_guess) - Gradient(old_guess)
    approx_hessian += ((np.outer(y_i, y_i)) / (np.dot(y_i, s_i))) - ((np.dot(approx_hessian,
                                                                             np.dot(np.outer(s_i, s_i), approx_hessian))) / (np.dot(s_i, np.dot(approx_hessian, s_i))))
    # update using Sherman-Morrison formula
    part_A = np.dot(s_i, y_i)
    part_A += np.dot(y_i, np.dot(inverse_approx_hessian, y_i))
    part_A /= np.dot(s_i, y_i)**2
    part_A *= np.outer(s_i, s_i)
    part_B = np.dot(inverse_approx_hessian, np.outer(y_i, s_i))
    part_B += np.dot(np.outer(s_i, y_i), inverse_approx_hessian)
    part_B /= np.dot(s_i, y_i)
    inverse_approx_hessian += part_A - part_B

    diff = old_guess - new_guess
    diff = np.sqrt(np.dot(diff, diff))
    print ("OLD GUESS: {}".format(old_guess))
    print ("NEW GUESS: {}".format(new_guess))
    print ("DIFFERENCE: {}".format(diff))
    old_guess = new_guess
    old_search_direction = new_search_direction
    steps_x.append(old_guess[0])
    steps_y.append(old_guess[1])
    if(diff < residual_tolerance):
        converged = True
steps_x = np.array(steps_x)
steps_y = np.array(steps_y)
steps_per_method["BFGS"] = num_steps
difference_per_method["BFGS"] = diff


In [ ]:
pf.plot_func(xminimum, xmaximum, yminimum, ymaximum, steps_x, steps_y, f)

# 4.5 What method to choose?

In [ ]:
print("{:->39}".format(""))
print("{:>20} : {:>5} {:>10}".format("Method", "Steps", "Difference"))
print("{:->20}   {:->5} {:->10}".format("", "", ""))
for i in sorted(difference_per_method, key=difference_per_method.get, reverse=True):
    print("{:>20} : {:>5} {:>6.4e}".format(i, steps_per_method[i], difference_per_method[i]))
print("{:->39}".format(""))


# 4.6 Application

In [ ]:
# DIMERS   
# radius = 4.5 # atomic units
# deformation = 0.65 # ratio (must be less than or equal to 1)
# RING
radius = 2.0 # atomic units
deformation = 1.0 # ratio (must be less than or equal to 1)



if deformation > 1.0 or deformation < 0.0:
        deformation = np.abs(deformation) % 1.0

steps_radius, steps_deformation, steps_energy = pf.optimize_H6_ring(radius, deformation)
        

In [ ]:
import ipywidgets as ipw
from ipywidgets import *
step_set = ipw.IntSlider(min=0,max =len(steps_energy)-1,value=0)
interactive_plot = ipw.interactive(pf.plotting, step=step_set,steps_radius=fixed(steps_radius), steps_energy=fixed(steps_energy),steps_deformation=fixed(steps_deformation))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

When deciding on an optimization method to use, we consider (above) the number of steps required to converge. The Newton-Raphson method converges quickly, and with great accuracy. However the Newton-Raphson method requires the calculation of a Hessian, which is expensive. The BFGS method avoids the Hessian calculation and instead iteratively builds an approximate Hessian. The BFGS method also converges quickly and is widely used in real world applications. The congujate gradient method requires more steps to converge, but is accurate. Finally, the steepest descent method, while slow, guarantees convergence.

# 4.6 Your Task

Optimization methods are commonly used in chemistry to find the lowest energy structure of a molecule, which corresponds to the equilibrium gas phase geometry. The basic approach is to use a computational method to evaluate the energy and gradients with respect to moves of the nuclei, and using this information with an optimization method to optimize the geometry. 

Given what you have learned about optimization methods:
What method(s) would you use to optimize the structure of a molecule?
Consider the expense/availability of 2nd derivatives.